In [ ]:
import os
import pylab
import string
import fdsreader
import matplotlib
import subprocess

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from importlib import reload
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
import spotpy
from scipy.stats.qmc import LatinHypercube
from pyDOE import lhs
import random
import shutil
import re

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import plot_tree, export_graphviz
import graphviz
from io import BytesIO
from PIL import Image

## Load Experiment data and merg parameters

In [ ]:
#LHC Data
#label_session="LHC_Session_2023-04-06_48"
#label_session="LHC_Session_2023-04-11_79"
#label_session="LHC_Session_2023-04-12_67"
#label_session="LHC_Session_2023-04-12_99"
label_session="LHC_Session_2023-04-13_94"


stor_id=label_session[12:]

sims_path_seed = os.path.join(f"../BurnerSims/{label_session}")
os.listdir(sims_path_seed)
stor_id

In [ ]:
# Path to experiment data
exp_root = os.path.join("../GeneralInformation/macfp-db")
exp_macfp_dir = os.path.join(exp_root+"/Fire_Growth/NIST_Parallel_Panel/Experimental_Data")

In [ ]:
# Load the table of the centerline - Burner_HF_Centerline_multi-layer.csv
centreline_hf_path = os.path.join(exp_macfp_dir, "Burner_HF_Centerline_multi-layer.csv")
centreline_hf_df = pd.read_csv(centreline_hf_path, header=0, skiprows=[1])

# Load the table ofthe centerline - Burner_steadyHF_Width_multi-layer.csv
ST_Width_layer_path = os.path.join(exp_macfp_dir, "Burner_steadyHF_Width_multi-layer.csv")
ST_Width_layer_df = pd.read_csv(ST_Width_layer_path, header=0, skiprows=[1])


In [ ]:
#Data of the experiment Center - time and Width - steady state
hf_TIME_center = centreline_hf_df[["HF_z20", "HF_z50", "HF_z75", "HF_z100"]].to_numpy()
hf_STEADY_width = ST_Width_layer_df[["HF_y-25", "HF_y-15", "HF_y0", "HF_y15", "HF_y25"]].to_numpy()

In [ ]:
# rebuild ranges for default
default_ranges = {}

# Open the file for reading
with open(f'{sims_path_seed}/param_set_ranges_{stor_id}.txt', 'r') as file:
    
    file.readline()

   # Iterate through the lines of the file
    for line in file:
        
        param_name, value1, value2, value3, value4 = [s.strip() for s in re.split(r':|,', line)]

        # Convert the values from string to float
        value1 = float(value1)
        value2 = float(value2)
        value3 = str(value3)
        value4 = str(value4)

        # Update the default_ranges dictionary
        default_ranges[param_name] = (value1, value2)

# Check 
print("Reconstructed dictionary:")
print(default_ranges)


In [ ]:
# merg parameter values of each case to one dataframe
TableData_path = "TableData"
output_csv = f"merged_param_values_{stor_id}.csv"

csv_files = [file for file in os.listdir(TableData_path) if file.startswith(label_session) and file.endswith("param_values.csv")]

merged_data = []

for csv_file in csv_files:
    # Extract the case number from the filename
    case_num = csv_file.split("_")[0][4:]

    # Read the CSV file
    df = pd.read_csv(os.path.join(TableData_path, csv_file))

    # Delete the existing "Case_sample" column if it exists
    if "Case_sample" in df.columns:
        del df["Case_sample"]
        
    # Add a new column for the case number and sample index
    df.insert(0, "Case_sample", df.index.map(lambda x: f"Case{case_num}_sample{x}"))

    # Append the dataframe to the merged_data list
    merged_data.append(df)

# Concatenate all the dataframes in the merged_data list
merged_df = pd.concat(merged_data, ignore_index=True)

# Save the merged dataframe to a new CSV file
merged_df.to_csv(f'{TableData_path}/{output_csv}', index=False)
LHC_sample_csv=pd.read_csv(f'{TableData_path}/{output_csv}', header=0)
len(LHC_sample_csv)

In [ ]:
LHC_sample_csv[0:10]

--------

## Process Simulation Results

In [ ]:
#some necessary output
devc=pd.read_csv(f"P:/Shared/Studium/Masterthesis/ParallelPanelBurnerSetup/BurnerSims/LHC_Session_{stor_id}/case0/MaCFP_CASE0_sample0/MaCFP_Burner_CASE0_sample0_devc.csv", header=0,skiprows=1)
param_values = devc.loc[:, devc.columns.str.startswith("Time")].values

#condition
search_entry=105
indexes = np.where(param_values > search_entry)

# first index that meets the condition
first_index = indexes[0][0] if len(indexes[0]) > 0 else None

# Simulation Time
sim_time=125
time_stepCSV=len(param_values)/sim_time


print("Total length:", len(param_values))
print("Timestep:", (time_stepCSV))
print(f"First index of entry higher than {search_entry}:", first_index) # to average over the last 20 s
print(f"Tail:", len(param_values)-first_index)

### SIM: Avg HF **CENTER** - Time

In [ ]:
# Define DEVC IDs and desired times
devc_ids = ["HF_y0_z20", "HF_y0_z50", "HF_y0_z75", "HF_y0_z100"]
desired_times = [20, 40, 60, 80]  # s

# Define time window to average over
frame_window = 3

# Create an empty dictionary to store the results
SIM_avg_HF_Center_dict = {}

# Loop through each case folder
for case_folder in os.listdir(sims_path_seed):
    if case_folder.startswith("case"):
        case_path = os.path.join(sims_path_seed, case_folder)
        case_num = case_folder.split("case")[-1]  # Extract the case number

        # Loop through each subfolder and read the CSV file
        for subdir in os.listdir(case_path):
            if subdir.startswith(f"MaCFP_CASE{case_num}_sample"):
                # Extract the sample number from the subfolder name
                sample_num = subdir.split("_")[-1]
                #print(sample_num)

                # Find the DEVC CSV file in the subfolder
                for file in os.listdir(os.path.join(case_path, subdir)):
                    if file.endswith("_devc.csv") and file.startswith(f"MaCFP_Burner_CASE{case_num}_sample") and file.endswith(f"{sample_num}_devc.csv"):
                        # Read the DEVC data from the CSV file
                        devc_data = pd.read_csv(os.path.join(case_path, subdir, file), header=1)


                        # Create an empty 2D array to store average flux values and MSEs
                        SIM_avg_Fluxes_center = np.zeros((len(devc_ids) + 1, len(desired_times)))

                        # Compute the average flux values for each DEVC ID and desired time
                        for i, devc_id in enumerate(devc_ids):
                            for j, desired_time in enumerate(desired_times):
                                # Compute time window to average over
                                frames = desired_time * 2
                                t_min = int(frames - frame_window)
                                t_max = int(frames + frame_window + 1)

                                # Compute average within the time window
                                flux_avrg = np.average(devc_data[devc_id][t_min:t_max].to_numpy())

                                # Store the average flux value in the array
                                SIM_avg_Fluxes_center[i, j] = flux_avrg

                        # Compute the mean squared error (MSE) and the Root Mean Squared Error (RMSE) for each desired time
                        for j, desired_time in enumerate(desired_times):
                                MSE = np.mean((SIM_avg_Fluxes_center[:-1, j] - hf_TIME_center[j]) ** 2)
                                SIM_avg_Fluxes_center[-1, j] = np.sqrt(MSE)
                         
                     #######   if int(sample_num[-1:]) < 10:
                            sample_num = "0" + sample_num
                        # Add the results to the dictionary using the sample number as the key
                        SIM_avg_HF_Center_dict[f'Case{case_num}_{sample_num}'] = SIM_avg_Fluxes_center

# Print the number of samples
print(len(SIM_avg_HF_Center_dict))

# Convert the  dictionary to a dataframe and save it to a CSV file
data = {key: value.flatten() for key, value in SIM_avg_HF_Center_dict.items()}
columns = []
for j, desired_time in enumerate(desired_times):
    for devc_id in devc_ids:
        columns.append(f"{devc_id}_T{desired_time}")
    columns.append(f"RMSE_T{desired_time}")

df = pd.DataFrame.from_dict(data, orient="index", columns=columns)
df.index.name = "sample_num"
df.to_csv(f"TableData/SIM_avg_HF_Center_{stor_id}.csv")


In [ ]:
pd.read_csv(f"TableData/SIM_avg_HF_Center_{stor_id}.csv", header=0)[0:10]

### SIM: **SURFACE** HF 

In [ ]:
# Define DEVC IDs
heights = ["z20", "z50", "z75", "z100"]
y_positions = ["-25", "-15", "0", "15", "25"]
devc_ids = [f"HF_y{y_pos}_{height}" for height in heights for y_pos in y_positions]

SIM_HF_surface = {}

# Loop through each case folder
for case_folder in os.listdir(sims_path_seed):
    if case_folder.startswith("case"):
        case_path = os.path.join(sims_path_seed, case_folder)
        case_num = case_folder.split("case")[-1]  # Extract the case number

        # Loop through each subfolder and read the CSV file
        for subdir in os.listdir(case_path):
            if subdir.startswith(f"MaCFP_CASE{case_num}_sample"):
                # Extract the sample number from the subfolder name
                sample_num = subdir.split("_")[-1]

                # Find the DEVC CSV file in the subfolder
                for file in os.listdir(os.path.join(case_path, subdir)):
                    if file.endswith("_devc.csv") and file.startswith(f"MaCFP_Burner_CASE{case_num}_sample") and file.endswith(f"{sample_num}_devc.csv"):
                        # Read the DEVC data from the CSV file
                        devc_data = pd.read_csv(os.path.join(case_path, subdir, file), header=1)
                        
                        # Compute the average for each devc_id over the last 20 s
                        devc_data_avg = devc_data[devc_ids].tail(58).mean()

                        # Reshape devc_data_avg to align with hf_STEADY_width
                        devc_data_avg_reshaped = np.array(devc_data_avg).reshape(len(heights), len(y_positions))

                        # Compute the mean squared error (MSE) and Root Mean Squared Error (RMSE) for the averaged data
                        
                        mse = ((devc_data_avg_reshaped - hf_STEADY_width) ** 2).mean(axis=1)
                        rmse = np.sqrt(mse)
                        
                        # Add the results to the dictionary using the case and the sample number as the key
                        SIM_HF_surface[f'Case{case_num}_{sample_num}'] = np.concatenate((devc_data_avg, rmse))
                        
# Print the number of samples
print(len(SIM_HF_surface))
# Convert the dictionary to a dataframe and save it to a CSV file
columns = devc_ids + [f'RMSE_{height}' for height in heights]
df = pd.DataFrame.from_dict(SIM_HF_surface, orient="index", columns=columns)
df.index.name = "sample_num"
df.sort_index(inplace=True) 
df.to_csv(f"TableData/SIM_HF_surface_{stor_id}.csv")

In [ ]:
pd.read_csv(f"TableData/SIM_HF_surface_{stor_id}.csv", header=0)[0:10]

### **HRR** of all Cases

In [ ]:
# Combine all HRR data to one file
# Create an empty dictionary to store the HRR
SIM_HRR = {}

# Loop through each case folder
for case_folder in os.listdir(sims_path_seed):
    if case_folder.startswith("case"):
        case_path = os.path.join(sims_path_seed, case_folder)
        # Extract the case number using regex
        match = re.search(r'\d+', case_folder)
        if match:
            case_num = match.group()
            # Loop through each subfolder and read the CSV file
            for subdir in os.listdir(case_path):
                # Extract the sample number using regex
                match = re.search(r'sample(\d+)', subdir)
                if match:
                    sample_num = match.group(1)
                    if subdir.startswith(f"MaCFP_CASE{case_num}_sample{sample_num}"):
                        # Find the HRR CSV file in the subfolder
                        for file in os.listdir(os.path.join(case_path, subdir)):
                            if file.endswith("_hrr.csv"):
                                # Read the HRR data from the CSV file
                                hrr_data = pd.read_csv(os.path.join(case_path, subdir, file), skiprows=[0], usecols=['HRR'])
                                # Create a new row with the HRR data and the column name
                                col_name = f"CASE{case_num}__sample{sample_num}"
                                new_row = pd.DataFrame({col_name: hrr_data['HRR'].values.flatten()})
                                SIM_HRR[f'Case{case_num}_{sample_num}'] = new_row

# Convert the dictionary to a dataframe and save it to a CSV file
SimHRR_data = pd.concat(SIM_HRR.values(), axis=1)                   
# Save the HRR data to a new CSV file
SimHRR_data.to_csv(f"{TableData_path}/hrr_data_{stor_id}.csv", index=False)
HRR_Sims=pd.read_csv(f'{TableData_path}/hrr_data_{stor_id}.csv')

print(len(HRR_Sims.columns))

In [ ]:
pd.read_csv(f"TableData/hrr_data_{stor_id}.csv", header=0)[0:10]

In [ ]:
# Load the HRR data
HRR_Sims = pd.read_csv(f'{TableData_path}/hrr_data_{stor_id}.csv')

# Plot all columns in one figure
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

# Plot each column
for col in HRR_Sims.columns[1:]:
    ax.plot(HRR_Sims[col], label=col)
    ax.set_title("HRR for all cases and samples")
    ax.set_xlabel("Index")
    ax.set_ylabel("HRR")
    

#plt.legend("")
plt.tight_layout()
plt.show()

**avg. HRRPUA** of all Cases

In [ ]:
# list to store the average HRR values for each case
HRRPUA_avg_lst = []
# to calc the HRRPUA
Burner_area=0.3*0.6
# Loop through each column and calculate the average HRR over the last 58 entries
for col in HRR_Sims.columns:
    avg_hrr = HRR_Sims[col].tail(58).mean()
    HRRPUA_avg_lst.append(avg_hrr/Burner_area)
    
HRRPUA_avg=np.array([HRRPUA_avg_lst])
print((HRRPUA_avg))

### **LHC Parameter** overview

### Distribution

In [ ]:
# Plot LHC Parameters for an overview
# Create a grid of plots with 2 columns
fig, axs = plt.subplots(nrows=len(LHC_sample_csv.columns), ncols=2, figsize=(12, 24))

# Analyze distribution and plot line for each column
for i, col in enumerate(LHC_sample_csv.columns[1:]):
    axs[i, 0].hist(LHC_sample_csv[col],bins=10)
    axs[i, 0].set_title(f"Distribution of {col}")
    axs[i, 0].set_xlabel(col)
    axs[i, 0].set_ylabel("Frequency")
    axs[i, 1].plot(LHC_sample_csv[col])
    axs[i, 1].set_title(col)
    axs[i, 1].set_xlabel("Index")
    axs[i, 1].set_ylabel(col)

plt.tight_layout()
plt.show()


### Check the Impact of the Parameters

### Random Forest Regression

Random Forest regression is an ensemble learning method that constructs multiple decision trees and combines their results to improve the overall predictive performance of the model. The method works well for both regression and classification tasks. In this case, we're using a Random Forest regression model to analyze the relationship between the input parameters and the RMSE values [[Towards Data Science](https://towardsdatascience.com/understanding-random-forest-58381e0602d2)].

The diagram below illustrates the basic structure of a Random Forest model:

![Random Forest](https://miro.medium.com/v2/resize:fit:640/format:webp/1*LMoJmXCsQlciGTEyoSN39g.jpeg)

*Image Source: [Towards Data Science](https://towardsdatascience.com/understanding-random-forest-58381e0602d2)*

1. The Random Forest algorithm constructs multiple decision trees by using a random selection of features and data points.
2. Each decision tree in the ensemble produces its own output (in this case, an RMSE value).
3. The final output of the Random Forest model is obtained by averaging the outputs of all the decision trees.

Example: a single decision tree from the Random Forest model

In [ ]:
# Get the file names for parameter data and simulation data
lhc_sess_param = [file for file in os.listdir(TableData_path) if file.startswith("merged_param_values") and file.endswith(".csv")]
sim_hf_sess = [file for file in os.listdir(TableData_path) if file.startswith("SIM_HF_surface") and file.endswith(".csv")]

all_rmse_values = []
all_lhc_dfs = []


# Loop through each "package" of simulations
for i in range(len(lhc_sess_param)):
    
    lhc_df = pd.read_csv(f'{TableData_path}/{lhc_sess_param[i]}', header=0)

    # Read the simulation data
    sim_hf_df = pd.read_csv(f'{TableData_path}/{sim_hf_sess[i]}', header=0)

    # Get the RMSE values
    rmse_values = sim_hf_df.loc[:, sim_hf_df.columns.str.startswith("RMSE")].values
    rmse_average = rmse_values.mean(axis=1)

    all_rmse_values.append(rmse_average)
    all_lhc_dfs.append(lhc_df)

# Combine the data from all "packages"
combined_rmse_values = np.concatenate(all_rmse_values)
combined_lhc_df = pd.concat(all_lhc_dfs)

# Get the unique parameter names
unique_params = [col for col in combined_lhc_df.columns if col != 'Case_sample']

# Get the parameter data
X = combined_lhc_df[unique_params].values

# Fit the Random Forest model
regr = RandomForestRegressor(n_estimators=100, random_state=0)
regr.fit(X, combined_rmse_values)

# Visualize a single decision tree from the Random Forest model

def visualize_tree(tree, feature_names, max_depth):
    dot_data = export_graphviz(tree, out_file=None, 
                               feature_names=feature_names,  
                               filled=True, rounded=True,  
                               special_characters=True,
                               max_depth=max_depth)  
    graph = graphviz.Source(dot_data) 
    graph.format = 'png' # Set the output format to PNG
    image_data = graph.pipe(format='png') # Get the binary image data as bytes
    image = Image.open(BytesIO(image_data)) # Create an Image object from the binary data
    return image

# Get the first decision tree from the Random Forest model
first_tree = regr.estimators_[0]

# Visualize the first decision tree with a maximum depth of 3
tree_image = visualize_tree(first_tree, unique_params, max_depth=4)
display(tree_image)

### Surface Anlaysis

In [ ]:
# Get the file names for parameter data and simulation data
lhc_sess_param = [file for file in os.listdir(TableData_path) if file.startswith("merged_param_values") and file.endswith(".csv")]
sim_hf_sess = [file for file in os.listdir(TableData_path) if file.startswith("SIM_HF_surface") and file.endswith(".csv")]

all_rmse_values = []
all_lhc_dfs = []

# Loop through each "package" of simulations
for i in range(len(lhc_sess_param)):
    # Read the parameter data
    lhc_df = pd.read_csv(f'{TableData_path}/{lhc_sess_param[i]}', header=0)

    # Read the simulation data
    sim_hf_df = pd.read_csv(f'{TableData_path}/{sim_hf_sess[i]}', header=0)

    # Get the RMSE values
    rmse_values = sim_hf_df.loc[:, sim_hf_df.columns.str.startswith("RMSE")].values
    rmse_average = rmse_values.mean(axis=1)

    all_rmse_values.append(rmse_average)
    all_lhc_dfs.append(lhc_df)

# Combine the data from all "packages"
combined_rmse_values = np.concatenate(all_rmse_values)
combined_lhc_df = pd.concat(all_lhc_dfs)

# Remove the 'Case_sample' column from the combined_lhc_df
combined_lhc_df = combined_lhc_df.drop(columns=['Case_sample'])

# Get the unique parameter names from the combined_lhc_df
unique_params = combined_lhc_df.columns.tolist()

# Create a Random Forest regression model
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model using the combined parameter sets and the combined RMSE values
X = combined_lhc_df[unique_params].values
y = combined_rmse_values
rf.fit(X, y)

# Calculate the feature importances
feature_importances = rf.feature_importances_

# Combine the parameter names and their importances into a DataFrame
importances_df = pd.DataFrame({'Parameter': unique_params, 'Importance': feature_importances})

# Sort the DataFrame based on the importances
sorted_importances_df = importances_df.sort_values(by='Importance', ascending=False)

# Print the sorted importances DataFrame
print(sorted_importances_df)

# Plot the feature importances
plt.figure(figsize=(22, 6))
plt.bar(sorted_importances_df['Parameter'], sorted_importances_df['Importance'])
plt.xlabel('Parameters')
plt.ylabel('Importance')
plt.title('Parameter Importance - Surface Anlaysis -')
plt.show()


### Time Analysis 

In [ ]:
# Get the file names for parameter data and simulation data
lhc_sess_param = [file for file in os.listdir(TableData_path) if file.startswith("merged_param_values") and file.endswith(".csv")]
sim_hf_sess_center = [file for file in os.listdir(TableData_path) if file.startswith("SIM_avg_HF_Center") and file.endswith(".csv")]

all_rmse_values = []
all_lhc_dfs = []

# Loop through each "package" of simulations
for i in range(len(lhc_sess_param)):
    # Read the parameter data
    lhc_df = pd.read_csv(f'{TableData_path}/{lhc_sess_param[i]}', header=0)

    # Read the simulation data
    sim_hf_df = pd.read_csv(f'{TableData_path}/{sim_hf_sess_center[i]}', header=0)

    # Get the RMSE values
    rmse_values = sim_hf_df.loc[:, sim_hf_df.columns.str.startswith("RMSE")].values
    rmse_average = rmse_values.mean(axis=1)

    all_rmse_values.append(rmse_average)
    all_lhc_dfs.append(lhc_df)

# Combine the data from all "packages"
combined_rmse_values = np.concatenate(all_rmse_values)
combined_lhc_df = pd.concat(all_lhc_dfs)

# Remove the 'Case_sample' column from the combined_lhc_df
combined_lhc_df = combined_lhc_df.drop(columns=['Case_sample'])

# Get the unique parameter names from the combined_lhc_df
unique_params = combined_lhc_df.columns.tolist()

# Create a Random Forest regression model
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model using the combined parameter sets and the combined RMSE values
X = combined_lhc_df[unique_params].values
y = combined_rmse_values
rf.fit(X, y)

# Calculate the feature importances
feature_importances = rf.feature_importances_

# Combine the parameter names and their importances into a DataFrame
importances_df = pd.DataFrame({'Parameter': unique_params, 'Importance': feature_importances})

# Sort the DataFrame based on the importances
sorted_importances_df = importances_df.sort_values(by='Importance', ascending=False)

# Print the sorted importances DataFrame
print(sorted_importances_df)

# Plot the feature importances
plt.figure(figsize=(22, 6))
plt.bar(sorted_importances_df['Parameter'], sorted_importances_df['Importance'])
plt.xlabel('Parameters')
plt.ylabel('Importance')
plt.title('Parameter Importance - Time Analysis -')
plt.show()


-----

# **Surface** anlaysis

see `Burner_steadyHF_Width_multi-layer.csv` of the [MaCFP/NIST parallel panel tests](https://github.com/MaCFP/macfp-db/tree/master/Fire_Growth/NIST_Parallel_Panel/Documentation).

Timsteps: steady state for device: HF_y-25, HF_y-15, HF_y0, HF_y15, HF_y25 with height: 20, 50, 75, 100

**Experiment Plot**

<img src="AssessBurnerOutput/BurnerPanelFluxMapExp.png" width="400" height="400">

### Overview

In [ ]:
sim_hf_df = pd.read_csv(f'{TableData_path}/SIM_HF_surface_{stor_id}.csv', header=0)
# Calculate the average RMSE for each iteration
rmse_values = sim_hf_df.loc[:, sim_hf_df.columns.str.startswith("RMSE")].values
rmse_average = rmse_values.mean(axis=1)

# Create a 1x2 grid of subplots for the line and bar plots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

# Line plot of the average RMSE data
axes[0].plot(rmse_average, marker='o', linestyle='-', linewidth=2)
axes[0].set_xlabel('Samples')
axes[0].set_ylabel('Average RMSE')
axes[0].set_title('Average RMSE over Samples (Line Plot)')
axes[0].grid(True,alpha=0.5)

# Bar plot of the average RMSE data
axes[1].bar(range(len(rmse_average)), rmse_average)
axes[1].set_xlabel('Samples')
axes[1].set_ylabel('Average RMSE')
axes[1].set_title('Average RMSE over Samples (Bar Plot)')
axes[1].grid(True,alpha=0.5)

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.3)

# Show the combined plot
plt.show()


### Correlation avg. HRRPUA and avg. RMSE 

In [ ]:
sim_hf_df = pd.read_csv(f'{TableData_path}/SIM_HF_surface_{stor_id}.csv', header=0)

# Extract the RMSE values and HRRPUA_avg
rmse_values = sim_hf_df.loc[:, sim_hf_df.columns.str.startswith("RMSE")].values
rmse_average = rmse_values.mean(axis=1)
HRRPUA_avg = HRRPUA_avg.ravel()

# Plot the scatter plot
plt.scatter(HRRPUA_avg, rmse_average)

# Add a trend line to the plot
z = np.polyfit(HRRPUA_avg, rmse_average, 1)
p = np.poly1d(z)
plt.plot(HRRPUA_avg,p(HRRPUA_avg),"r--")

# Add axis labels and a title
plt.xlabel('avg. HRRPUA')
plt.ylabel('avg. RMSE')
plt.title('Surface (width): Correlation between HRRPUA_avg and RMSE Average')

plt.show()


### HRRPUA vs. Parameter

In [ ]:
# Read the CSV files
lhc_df = pd.read_csv(f'{TableData_path}/{output_csv}', header=0)
sim_hf_df = pd.read_csv(f'{TableData_path}/SIM_HF_surface_{stor_id}.csv', header=0)

# Extract the parameter values, HRRPUA_avg, and RMSE values
param = "TMPA"
rmse_values = sim_hf_df.loc[:, sim_hf_df.columns.str.startswith("RMSE")].values
rmse_average = rmse_values.mean(axis=1)
param_values = lhc_df.loc[:, lhc_df.columns.str.startswith(param)].values.ravel()
HRRPUA_avg = HRRPUA_avg.ravel()

# Get the first column of lhc_df as the labels
labels = lhc_df.iloc[:, 0].values

# Combine the labels, HRRPUA_avg, param_values, and rmse_average
data_dict = {
    i: {
        "case_id": lhc_df.iloc[i, 0],
        "HRRPUA_avg": HRRPUA_avg[i],
        "param_value": param_values[i],
        "rmse_average": rmse_average[i],
    }
    for i in range(len(HRRPUA_avg))
}


# Calculate the min and max values of the colorbar based on the rmse_average values
rmse_range = np.max(rmse_average) - np.min(rmse_average)
cbar_min = np.min(rmse_average) - 0.1 * rmse_range
cbar_max = np.max(rmse_average) + 0.1 * rmse_range

# Create a colormap for the rmse_average range
norm = plt.Normalize(vmin=cbar_min, vmax=cbar_max)
cmap = plt.get_cmap("jet")

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_xlabel("avg. HRRPUA")
ax.set_ylabel(param)
ax.set_title(f"Surface (width): avg. HRRPUA vs {param}")

# Plot each entry in the data_dict as a scatter plot point
for label, data in data_dict.items():
    hrrpua_avg = data["HRRPUA_avg"]
    param_value = data["param_value"]
    rmse_average = data["rmse_average"]
    color = cmap(norm(rmse_average))
    ax.scatter(hrrpua_avg, param_value, c=[color])

# Add a colorbar to represent the rmse_average range
cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap))
cbar.set_label('avg. RMSE')

plt.show()


-------

# **Time** Analysis 

see `Burner_HF_Centerline_multi-layer.csv` of the [MaCFP/NIST parallel panel tests](https://github.com/MaCFP/macfp-db/tree/master/Fire_Growth/NIST_Parallel_Panel/Documentation).

Timsteps: 20,40,60,80 for device: HF_z20, HF_z50, HF_z75, HF_z100

**Experiment Plot**

<img src="AssessBurnerOutput/BurnerPanelCentreLineHeatFluxExp.png" width="600" height="400">

### Overview

In [ ]:
sim_hf_df_Center = pd.read_csv(f"TableData/SIM_avg_HF_Center_{stor_id}.csv", header=0)
# Calculate the average RMSE for each iteration
rmse_values = sim_hf_df_Center.loc[:, sim_hf_df_Center.columns.str.startswith("RMSE")].values
rmse_average = rmse_values.mean(axis=1)

# Create a 1x2 grid of subplots for the line and bar plots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

# Line plot of the average RMSE data
axes[0].plot(rmse_average, marker='o', linestyle='-', linewidth=2)
axes[0].set_xlabel('Samples')
axes[0].set_ylabel('Average RMSE')
axes[0].set_title('Average RMSE over Samples (Line Plot)')
axes[0].grid(True,alpha=0.5)

# Bar plot of the average RMSE data
axes[1].bar(range(len(rmse_average)), rmse_average)
axes[1].set_xlabel('Samples')
axes[1].set_ylabel('Average RMSE')
axes[1].set_title('Average RMSE over Samples (Bar Plot)')
axes[1].grid(True,alpha=0.5)

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.3)

# Show the combined plot
plt.show()


### Correlation avg. HRRPUA and avg. RMSE 

In [ ]:
sim_hf_df_Center = pd.read_csv(f"TableData/SIM_avg_HF_Center_{stor_id}.csv", header=0)

# Extract the RMSE values and HRRPUA_avg
rmse_values = sim_hf_df_Center.loc[:, sim_hf_df_Center.columns.str.startswith("RMSE")].values
rmse_average = rmse_values.mean(axis=1)
HRRPUA_avg = HRRPUA_avg.ravel()

# Plot the scatter plot
plt.scatter(HRRPUA_avg, rmse_average)

# Add a trend line to the plot
z = np.polyfit(HRRPUA_avg, rmse_average, 1)
p = np.poly1d(z)
plt.plot(HRRPUA_avg,p(HRRPUA_avg),"r--")

# Add axis labels and a title
plt.xlabel('avg. HRRPUA')
plt.ylabel('avg. RMSE')
plt.title('Center (Time): Correlation between HRRPUA_avg and RMSE Average')

plt.show()


### HRRPUA vs. Parameter

In [ ]:
# Read the CSV files
lhc_df = pd.read_csv(f'{TableData_path}/{output_csv}', header=0)
sim_hf_df_Center = pd.read_csv(f"TableData/SIM_avg_HF_Center_{stor_id}.csv", header=0)

# Extract the parameter values, HRRPUA_avg, and RMSE values
param = "TMPA"
rmse_values = sim_hf_df_Center.loc[:, sim_hf_df_Center.columns.str.startswith("RMSE")].values
rmse_average = rmse_values.mean(axis=1)
param_values = lhc_df.loc[:, lhc_df.columns.str.startswith(param)].values.ravel()
HRRPUA_avg = HRRPUA_avg.ravel()

# Get the first column of lhc_df as the labels
labels = lhc_df.iloc[:, 0].values

# Combine the labels, HRRPUA_avg, param_values, and rmse_average
data_dict = {
    i: {
        "case_id": lhc_df.iloc[i, 0],
        "HRRPUA_avg": HRRPUA_avg[i],
        "param_value": param_values[i],
        "rmse_average": rmse_average[i],
    }
    for i in range(len(HRRPUA_avg))
}


# Calculate the min and max values of the colorbar based on the rmse_average values
rmse_range = np.max(rmse_average) - np.min(rmse_average)
cbar_min = np.min(rmse_average) - 0.1 * rmse_range
cbar_max = np.max(rmse_average) + 0.1 * rmse_range

# Create a colormap for the rmse_average range
norm = plt.Normalize(vmin=cbar_min, vmax=cbar_max)
cmap = plt.get_cmap("jet")

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_xlabel("avg. HRRPUA")
ax.set_ylabel(param)
ax.set_title(f"Center (Time): avg. HRRPUA vs {param}")

# Plot each entry in the data_dict as a scatter plot point
for label, data in data_dict.items():
    hrrpua_avg = data["HRRPUA_avg"]
    param_value = data["param_value"]
    rmse_average = data["rmse_average"]
    color = cmap(norm(rmse_average))
    ax.scatter(hrrpua_avg, param_value, c=[color])

# Add colorbar 
cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap))
cbar.set_label('avg. RMSE')

plt.show()

-----------------------------------------

### Adjusting Range: pymoo - Multi-objective Optimization

<div class="alert alert-block alert-info">
<b>Ref.:</b> J. Blank and K. Deb, "Pymoo: Multi-Objective Optimization in Python," in IEEE Access, vol. 8, pp. 89497-89509, 2020, doi: <a href="https://ieeexplore.ieee.org/document/9078759">10.1109./ACCESS.2020.2990567</a>.
</div>

The **Genetic Algorithm (GA)** is a optimization technique inspired by the process of natural selection and evolution. It works by iteratively evolving a population of candidate solutions towards an optimal solution. The GA has several key components, including selection, crossover, and mutation, which work together to explore and exploit the solution space effectively.
<a href="https://pymoo.org/algorithms/soo/ga.html">[1]</a>


* 1. Initialization: Generate an initial population of candidate solutions randomly or using a sampling method. Each solution is called an individual and is represented as a chromosome, which encodes the decision variables.

* 2. Evaluation: Calculate the fitness value for each individual in the population using the objective function(s). In single-objective optimization, the fitness value is often the same as the objective value.

* 3. Selection: Select a subset of individuals from the population to act as parents for the next generation. Selection is usually based on fitness values, with better-performing individuals having a higher probability of being chosen. This process is known as fitness-proportional or roulette-wheel selection. Other selection methods include tournament selection, rank-based selection, and truncation selection.

* 4. Crossover: Apply the crossover (also called recombination) operator to pairs of selected parents to generate offspring. Crossover combines the genetic material from two parents to create one or more offspring, which inherit features from both parents. There are various crossover techniques, such as one-point, two-point, or uniform crossover, depending on the representation of the chromosomes.

* 5. Mutation: Apply the mutation operator to the offspring with a certain probability. Mutation introduces small random changes to the offspring's genetic material, promoting diversity in the population and helping the search algorithm explore the solution space more effectively. Mutation operators depend on the representation of the chromosomes and include bit-flip mutation for binary strings, swap mutation for permutations, and Gaussian mutation for real-valued decision variables.

* 6. Replacement: Replace some or all individuals in the current population with the newly generated offspring. Replacement strategies include generational replacement, where the entire population is replaced, and steady-state replacement, where only a portion of the population is replaced.

* 7. Termination: Check if a termination criterion has been met, such as reaching a maximum number of generations, achieving a desired fitness value, or not observing significant improvements in the best fitness value for a certain number of generations. If the termination criterion is not met, return to step 2 (evaluation).

In [ ]:
# Dummy data for demonstration purposes
population = np.random.rand(50, 2)
fitness_values = np.random.rand(50)
selection_counts = np.random.randint(1, 10, 50)
parent1, parent2 = np.random.rand(10), np.random.rand(10)
offspring_before_mutation, offspring_after_mutation = np.random.rand(10), np.random.rand(10)
population_next_gen = np.random.rand(50, 2)
best_fitness = np.random.rand(50)

fig, axes = plt.subplots(2, 4, figsize=(18, 8))

# 1. Initialization
axes[0, 0].scatter(population[:, 0], population[:, 1])
axes[0, 0].set_title("1. Initialization")
axes[0, 0].set_xlabel("Variable 1")
axes[0, 0].set_ylabel("Variable 2")

# 2. Evaluation
axes[0, 1].plot(fitness_values, marker="o", linestyle="")
axes[0, 1].set_title("2. Evaluation")
axes[0, 1].set_xlabel("Individual Index")
axes[0, 1].set_ylabel("Fitness Value")

# 3. Selection
axes[0, 2].bar(np.arange(len(selection_counts)), selection_counts)
axes[0, 2].set_title("3. Selection")
axes[0, 2].set_xlabel("Individual Index")
axes[0, 2].set_ylabel("Selection Count")

# 4. Crossover
parent1_plot, = axes[0, 3].plot(parent1, color="blue", label="Parent 1")
parent2_plot, = axes[0, 3].plot(parent2, color="red", label="Parent 2")
axes[0, 3].set_title("4. Crossover")
axes[0, 3].set_xlabel("Chromosome Index")
axes[0, 3].set_ylabel("Decision Variable")
axes[0, 3].legend()

# 5. Mutation
before_mutation_plot, = axes[1, 0].plot(offspring_before_mutation, linestyle="--", color="blue", label="Before Mutation")
after_mutation_plot, = axes[1, 0].plot(offspring_after_mutation, linestyle="-", color="red", label="After Mutation")
axes[1, 0].set_title("5. Mutation")
axes[1, 0].set_xlabel("Chromosome Index")
axes[1, 0].set_ylabel("Decision Variable")
axes[1, 0].legend()

# 6. Replacement
axes[1, 1].scatter(population[:, 0], population[:, 1], marker="o", label="Before Replacement")
axes[1, 1].scatter(population_next_gen[:, 0], population_next_gen[:, 1], marker="x", label="After Replacement")
axes[1, 1].set_title("6. Replacement")
axes[1, 1].set_xlabel("Variable 1")
axes[1, 1].set_ylabel("Variable 2")
axes[1, 1].legend()

# 7. Termination
axes[1, 2].plot(best_fitness)
axes[1, 2].set_title("7. Termination")
axes[1, 2].set_xlabel("Generation")
axes[1, 2].set_ylabel("Best Fitness Value")

# Remove the last subplot
axes[1, 3].axis("off")

plt.tight_layout()
plt.show()


The **GA algorithm** is configured with these parameters:
* pop_size=100: A population size of 100 individuals, which balances exploration of the solution space and computational cost.
* n_offsprings=50: 50 offspring are generated in each generation, balancing convergence speed and computational cost.
* sampling=FloatRandomSampling(): FloatRandomSampling is used for the initial population generation, which generates random floating-point numbers <a href="https://pymoo.org/operators/sampling.html">[2]</a>.
    * FloatRandomSampling creates the initial population by generating random floating-point numbers for each decision variable within their respective lower and upper bounds. This ensures that the search process starts from various regions in the solution space, allowing the algorithm to explore different combinations of decision variable values and potentially find better solutions.
* crossover=SBX(prob=0.9, eta=20): Simulated Binary Crossover (SBX) is employed with a probability of 0.9 and an eta value of 20, controlling exploration versus exploitation during crossover.
    * SBX operates on two parent solutions to generate two offspring solutions by combining their decision variables. The key idea is to create offspring that are statistically distributed around the parents, similar to the way single-point crossover works with binary strings <a href="https://content.wolfram.com/uploads/sites/13/2018/02/09-2-2.pdf">[3]</a>  <a href="https://dl.acm.org/doi/10.1145/1276958.1277190">[4]</a>  <a href="https://pymoo.org/operators/crossover.html?highlight=simulated%20binary%20crossover%20sbx">[5]</a>.The SBX operator works as follows:
        * Select two parent solutions from the current population.
        * For each decision variable, generate a random number 'u' between 0 and 1.
        * Calculate the blending factor 'β' using the random number 'u', a distribution index 'η' (eta), and the corresponding decision variables of the parents.
        * Create two offspring solutions by combining the parent decision variables using the blending factor 'β'.
    * The distribution index 'η' controls the exploration-exploitation trade-off during crossover. A larger 'η' value results in offspring that are closer to their parents (exploitation), while a smaller 'η' value generates offspring that are more spread out in the search space (exploration).
* mutation=PolynomialMutation(prob=0.1, eta=20): Polynomial Mutation is used with a probability of 0.1 and an eta value of 20, controlling mutation step size and exploration of the search space.
    * Polynomial Mutation operates on a single solution and modifies its decision variables <a href="https://dl.acm.org/doi/10.1145/1276958.1277190">[4]</a> <a href=" https://pymoo.org/operators/mutation.html?highlight=polynomialmutation">[6]</a>. The mutation operator works as follows:
      * Select a solution from the current population.
      * For each decision variable in the solution, generate a random number 'u' between 0 and 1.
      * Calculate the mutation scaling factor 'δ' using the random number 'u', a distribution index 'η' (eta), and the variable's lower and upper bounds.
      * Add the mutation scaling factor 'δ' to the decision variable to create the mutated variable.
The distribution index 'η' controls the mutation step size and exploration of the search space. A larger 'η' value results in smaller mutation steps, generating offspring closer to their parent (exploitation), while a smaller 'η' value generates offspring further away from their parent (exploration).
* eliminate_duplicates=True: This setting ensures that duplicate solutions are removed, maintaining diversity in the population.

Further:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.problems import get_problem
from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PolynomialMutation
The **CustomProblem** takes two arguments: rmse_normalized and param_values_normalized. It calls the parent class constructor (Problem) with the following specifications:

* n_var=1: The problem has 1 decision variable, corresponding to the index of the parameter values array.
* n_obj=1: There is a single objective, which is to minimize the normalized RMSE.
* n_constr=0: There are no constraints in this problem.
* xl=np.array([0]) and xu=np.array([1]): The lower and upper bounds for the decision variable are 0 and 1, respectively.
* it also stores the input rmse_normalized and param_values_normalized as class attributes for later use in the evaluation function

**_evaluate method** is responsible for evaluating the objective function. It takes the input decision variable x and calculates the corresponding objective value (normalized RMSE) based on the stored rmse_normalized data. 
* First, it converts the input x to indices by scaling it with the length of the rmse_normalized array and rounding the result. 
* Then, it assigns the objective value out["F"] by indexing the rmse_normalized array with the calculated indices. 
* This process allows the optimization algorithm to search for the optimal parameter values that minimize the RMSE.

In [ ]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.problems import get_problem
from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PolynomialMutation

### **Surface** anlaysis Parameter Opti

In [ ]:
# Define the custom problem class
class CustomProblem(Problem):
    def __init__(self, rmse_normalized, param_values_normalized):
        # Call the parent class constructor with problem specifications
        super().__init__(n_var=1,
                         n_obj=1,
                         n_constr=0,
                         xl=np.array([0]),
                         xu=np.array([1]))

        # Store the normalized RMSE and parameter values as class attributes
        self.rmse_normalized = rmse_normalized
        self.param_values_normalized = param_values_normalized

    # Define the evaluation function
    def _evaluate(self, x, out, *args, **kwargs):
        # Convert the input x to indices
        idx = np.round(x * (len(self.rmse_normalized) - 1)).astype(int)

        # Calculate the objective values for the given indices
        out["F"] = self.rmse_normalized[idx]

# Load the data from the CSV files
lhc_df = pd.read_csv(f'{TableData_path}/{output_csv}', header=0)
sim_hf_df = pd.read_csv(f'{TableData_path}/SIM_HF_surface_{stor_id}.csv', header=0)

# Calculate the average RMSE and HRRPUA values
rmse_values = sim_hf_df.loc[:, sim_hf_df.columns.str.startswith("RMSE")].values
rmse_average = rmse_values.mean(axis=1)

# Normalize the RMSE and HRRPUA values
rmse_normalized = (rmse_average - rmse_average.min()) / (rmse_average.max() - rmse_average.min())

# Get the unique parameters from the lhc_df
unique_params = lhc_df.columns.unique()[1:]

# Initialize lists to store results and best values
results_list = []
best_values_list = []

# Loop through each parameter and optimize using GA
for param in unique_params:
    # Get the parameter values and normalize them
    param_values = lhc_df.loc[:, lhc_df.columns.str.startswith(param)].values.ravel()
    param_values_normalized = (param_values - param_values.min()) / (param_values.max() - param_values.min())

    # Create a custom problem instance for the current parameter
    problem = CustomProblem(rmse_normalized, param_values_normalized)

    # Define the GA algorithm with the specified parameters
    algorithm = GA(
        pop_size=100,
        n_offsprings=50,
        sampling=FloatRandomSampling(),
        crossover=SBX(prob=0.9, eta=20),
        mutation=PolynomialMutation(prob=0.1, eta=20),
        eliminate_duplicates=True,
    )

    # Perform the optimization
    res = minimize(
        problem,
        algorithm,
        ("n_gen", 50),
        verbose=False,
    )

    pareto_front = res.F
    best_solution = res.X[np.argmin(res.F)]
    best_value = param_values[int(np.round(best_solution * (len(param_values) - 1)))]
    ###Add x % to min and max 
    best_value_minus_10 = best_value - 0.01 * best_value
    best_value_plus_10 = best_value + 0.01 * best_value
    ###Clip if default is reached
    lower_bound, upper_bound = default_ranges[param]
    clipped_lower_bound = np.clip(best_value_minus_10, lower_bound, upper_bound)
    clipped_upper_bound = np.clip(best_value_plus_10, lower_bound, upper_bound)

    best_values_list.append((param, clipped_lower_bound, clipped_upper_bound))

output1=[]
# Print the best values and their clipped ranges
print("\nBest values and their clipped ranges:")
for best_value_info in best_values_list:
    print(best_value_info)
    output1.append(best_value_info)



### **Time** Analysis Parameter Opti

In [ ]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.problems import get_problem
from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PolynomialMutation

# Define the custom problem class
class CustomProblem(Problem):
    def __init__(self, rmse_normalized, param_values_normalized):
        # Call the parent class constructor with problem specifications
        super().__init__(n_var=1,
                         n_obj=1,
                         n_constr=0,
                         xl=np.array([0]),
                         xu=np.array([1]))

        # Store the normalized RMSE and parameter values as class attributes
        self.rmse_normalized = rmse_normalized
        self.param_values_normalized = param_values_normalized

    # Define the evaluation function
    def _evaluate(self, x, out, *args, **kwargs):
        # Convert the input x to indices
        idx = np.round(x * (len(self.rmse_normalized) - 1)).astype(int)

        # Calculate the objective values for the given indices
        out["F"] = self.rmse_normalized[idx]

# Load the data from the CSV files
lhc_df = pd.read_csv(f'{TableData_path}/{output_csv}', header=0)
sim_hf_df = pd.read_csv(f'{TableData_path}/SIM_avg_HF_Center_{stor_id}.csv', header=0)

# Calculate the average RMSE and HRRPUA values
rmse_values = sim_hf_df.loc[:, sim_hf_df.columns.str.startswith("RMSE")].values
rmse_average = rmse_values.mean(axis=1)

# Normalize the RMSE and HRRPUA values
rmse_normalized = (rmse_average - rmse_average.min()) / (rmse_average.max() - rmse_average.min())

# Get the unique parameters from the lhc_df
unique_params = lhc_df.columns.unique()[1:]

# Initialize lists to store results and best values
results_list = []
best_values_list = []

# Loop through each parameter and optimize using GA
for param in unique_params:
    # Get the parameter values and normalize them
    param_values = lhc_df.loc[:, lhc_df.columns.str.startswith(param)].values.ravel()
    param_values_normalized = (param_values - param_values.min()) / (param_values.max() - param_values.min())

    # Create a custom problem instance for the current parameter
    problem = CustomProblem(rmse_normalized, param_values_normalized)

    # Define the GA algorithm with the specified parameters
    algorithm = GA(
        pop_size=100,
        n_offsprings=50,
        sampling=FloatRandomSampling(),
        crossover=SBX(prob=0.9, eta=20),
        mutation=PolynomialMutation(prob=0.1, eta=20),
        eliminate_duplicates=True,
    )

    # Perform the optimization 
    res = minimize(
        problem,
        algorithm,
        ("n_gen", 50),
        verbose=False,
    )

    pareto_front = res.F
    best_solution = res.X[np.argmin(res.F)]
    best_value = param_values[int(np.round(best_solution * (len(param_values) - 1)))]
    ###Add x % to min and max 
    best_value_minus_10 = best_value - 0.01 * best_value
    best_value_plus_10 = best_value + 0.01 * best_value
    ###Clip if default is reached
    lower_bound, upper_bound = default_ranges[param]
    clipped_lower_bound = np.clip(best_value_minus_10, lower_bound, upper_bound)
    clipped_upper_bound = np.clip(best_value_plus_10, lower_bound, upper_bound)

    best_values_list.append((param, clipped_lower_bound, clipped_upper_bound))

output2=[]
# Print the best values and their clipped ranges
print("\nBest values and their clipped ranges:")
for best_value_info in best_values_list:
    print(best_value_info)
    output2.append(best_value_info)



### Compare the two outputs to creat new Ranges for the next LHC Session

In [ ]:
#output can be copied to LHS_FDS...ipynb in params_info cell
combined_ranges = []

for param1, param2 in zip(output1, output2):
    param_name, min1, max1 = param1
    _, min2, max2 = param2

    combined_min = min(min1, min2)
    combined_max = max(max1, max2)

    # Add the additional "values" for each parameter
    if param_name == 'PATH_LENGTH':
        lhs_method = 'simple'
        param_type = None
    elif param_name == 'ANGLE_INCREMENT':
        lhs_method = 'simple'
        param_type = None
    elif param_name == 'NUMBER_RADIATION_ANGLES':
        lhs_method = 'LHS'
        param_type = None
    elif param_name == 'HRRPUA':
        lhs_method = 'LHS'
        param_type = None
    elif param_name == 'SOOT_YIELD':
        lhs_method = 'LHS'
        param_type = None
    elif param_name == 'RADIATIVE_FRACTION':
        lhs_method = 'LHS'
        param_type = None
    elif param_name == 'TMPA':
        lhs_method = 'LHS'
        param_type = None
    elif param_name == 'EMISSIVITY_Burner':
        lhs_method = 'LHS'
        param_type = 'random'
    elif param_name == 'EMISSIVITY_Panel':
        lhs_method = 'LHS'
        param_type = 'random'
    else:
        lhs_method = None
        param_type = None

    combined_ranges.append((param_name, combined_min, combined_max, lhs_method,param_type))

for param_range in combined_ranges:
    print(f'{param_range},')


### History of Parameter Sets

In [ ]:
# Define the path to the directory containing the text files
path_over = "../BurnerSims/"

# Define a dictionary to store the parameter ranges for each text file
param_ranges_txt = {}

# Loop through each text file
for folder in os.listdir(path_over):
    if folder.startswith(label_session[:-14]):
        folder_path = os.path.join(path_over, folder)
        for data in os.listdir(folder_path):
            if data.startswith("param_set_ranges_"):
                text_path = os.path.join(folder_path, data)

                # Open the file for reading
                with open(text_path, 'r') as file:
                    file.readline()

                    for line in file:
                        param_name, value1, value2, value3, value4 = [s.strip() for s in re.split(r':|,', line)]

                        # Convert the values from string to float
                        value1 = float(value1)
                        value2 = float(value2)

                        # Add the parameter range 
                        if param_name not in param_ranges_txt:
                            param_ranges_txt[param_name] = []
                        param_ranges_txt[param_name].append((value1, value2))

# DataFrame to store the parameter ranges 
pd.DataFrame(param_ranges_txt).T
